In [1]:
# remover weekday da tabela, deixar la so de leitura
# o cara nao abasteceu e ruptura loja

In [1]:
import pandas as pd
from ruptura.RupturaTable import RupturaTable
from ruptura.CreateBatch import CreateBatch
import collections
from ruptura.EncoderX import EncoderX

In [2]:
rupturaTable = RupturaTable()
files = ['barbieri-only.csv']  #concat all files here
clipData = ['01/11/2018', '01/04/2019']
rupturaTable.generate(files,clipData)

In [3]:
%%time
createBatch = CreateBatch('0-1-0')
amostras = createBatch.create(rupturaTable.getData(), 'BARBIERI', '01/03/2019')
#createBatch.exportBatch(amostras, 'barbieri-ymod.json')

Wall time: 18.2 s


In [5]:
amostras['Mart Minas Paracatu Loja 224-BISCOITO BARBIERI ESCALDADO 150G']

{'x': [{'Ocorr ncia': 'Reposi o',
   'Ocorr ncia2': 'Unidades Estocadas Armazenadas',
   'Data Hora de encerramento da OS': '01/12/2018 09:22'},
  {'Ocorr ncia': 'Reposi o',
   'Ocorr ncia2': 'Sem Unidades Estocadas Armazenadas',
   'Data Hora de encerramento da OS': '08/12/2018 11:24'},
  {'Ocorr ncia': 'Reposi o',
   'Ocorr ncia2': 'Sem Unidades Estocadas Armazenadas',
   'Data Hora de encerramento da OS': '12/12/2018 11:45'},
  {'Ocorr ncia': 'Presen a',
   'Ocorr ncia2': 'Sem Unidades Estocadas Armazenadas',
   'Data Hora de encerramento da OS': '13/12/2018 16:30'},
  {'Ocorr ncia': 'Reposi o',
   'Ocorr ncia2': 'Sem Unidades Estocadas Armazenadas',
   'Data Hora de encerramento da OS': '20/12/2018 16:31'},
  {'Ocorr ncia': 'Presen a',
   'Ocorr ncia2': 'Sem Unidades Estocadas Armazenadas',
   'Data Hora de encerramento da OS': '23/12/2018 10:04'},
  {'Ocorr ncia': 'Presen a',
   'Ocorr ncia2': 'Sem Unidades Estocadas Armazenadas',
   'Data Hora de encerramento da OS': '23/12/2018 

In [6]:
# id - produto
# score_date - data de referencia da proxima visita
# x1, y1... informacoes da visita mais recente
# x2, y2... informacaes da segunda visita mais recente
# ...



In [4]:
import datetime
from ruptura.TableEnrichment import TableEnrichment

# dado uma data qualquer ---> pra isso tem que transformar esse cara (-160 e um numero conheciudo)
# pegar as ultimas N visitas para um determinado produto (n=5,10 etc.)
# contruir um X assim [x1,data1, x2,data2, x3,data3 ... ]
def dateStringToDatetime(dateI):
    momento = dateI.split('/')
    dia = int(momento[0])
    mes = int(momento[1])
    ano = int(momento[2].split(' ')[0])
    return datetime.datetime(ano, mes, dia)

def datetimeToDateString(dateI):
    if dateI.day < 10:
        day = '0' + str(dateI.day)
    else:
        day = str(dateI.day)
    if dateI.month < 10:
        month = '0' + str(dateI.month)
    else:
        month = str(dateI.month)
    return day + '/' + month + '/' + str(dateI.year)

In [5]:
tableEnrich = TableEnrichment('0-2-1')
visits = tableEnrich.generate(amostras)
visits['Mart Minas Paracatu Loja 224']

{'01/12/2018': {'BISCOITO BARBIERI ESCALDADO 150G': {'Ocorr ncia': 'Reposi o',
   'Ocorr ncia2': 'Unidades Estocadas Armazenadas',
   'Data Hora de encerramento da OS': '01/12/2018 09:22',
   'status': 'present'},
  'BISCOITO BARBIERI PAPA OVO 150G': {'Ocorr ncia': 'Reposi o',
   'Ocorr ncia2': 'Unidades Estocadas Armazenadas',
   'Data Hora de encerramento da OS': '01/12/2018 09:22',
   'status': 'present'},
  'BISCOITO BARBIERI POLVILHO 250G': {'Ocorr ncia': 'Reposi o',
   'Ocorr ncia2': 'Unidades Estocadas Armazenadas',
   'Data Hora de encerramento da OS': '01/12/2018 09:22',
   'status': 'present'},
  'BISCOITO BARBIERI POLVILHO ARGOLA TRADICIONAL 150G': {'Ocorr ncia': 'Reposi o',
   'Ocorr ncia2': 'Unidades Estocadas Armazenadas',
   'Data Hora de encerramento da OS': '01/12/2018 09:22',
   'status': 'present'},
  'BISCOITO BARBIERI POLVILHO PALITO SABOR QUEIJO 150G': {'Ocorr ncia': 'Reposi o',
   'Ocorr ncia2': 'Unidades Estocadas Armazenadas',
   'Data Hora de encerramento da O

In [6]:
from heapq import nsmallest
from heapq import nlargest

def dateRefStore(visitStore, referenceWeek):
    dateRefStore = {}
    for refDate in referenceWeek:
        if refDate in visitStore:
            dateRefStore['ref'] = []
            dateRefStore['date'] = []
            for date in visits[store]:
                if date == 'info':
                    continue
                dateRefStore['ref'].append((dateStringToDatetime(date) - dateStringToDatetime(refDate)).days)
                dateRefStore['date'].append(date)
            dateRefStore['date_score'] = refDate
            return dateRefStore
    return dateRefStore

def dateRefStoreByDate(visitStore, refDate):
    dateRefStore = {}
    if refDate in visitStore:
        dateRefStore['ref'] = []
        dateRefStore['date'] = []
        for date in visits[store]:
            if date == 'info':
                continue
            dateRefStore['ref'].append((dateStringToDatetime(date) - dateStringToDatetime(refDate)).days)
            dateRefStore['date'].append(date)
        dateRefStore['date_score'] = refDate
        return dateRefStore
    return dateRefStore


def removePositiveValues(refList):
    if len(refList) == 0:
        return []
    maxValue = max(refList)
    maxIndex = refList.index(maxValue)
    if maxValue > 0:
        refList.pop(maxIndex)
        return removePositiveValues(refList) #recursive removal
    else:
        return refList

def remove0Value(refList):
    if 0 in refList:
        zeroIndex = refList.index(0)
        refList.pop(zeroIndex)
        return [True, refList]
    else:
        return [False, refList]

def findLastVisits(dateRef):
    NlastVisits = 5
    refList = list(dateRef['ref'])
    refList = removePositiveValues(refList)
    wasRemoved, refList = remove0Value(refList) #colocar esse cara como saida no fim
    nLastVisits = nlargest(NlastVisits, refList) # conferir se o numero e o mesmo pedido pq se nao pode ser importante adicionar um padding aqui.
    nLastVisits.sort(reverse=True)
    nLastVisitsDate = []
    for ref in nLastVisits:
        refIndex = dateRef['ref'].index(ref)
        nLastVisitsDate.append(dateRef['date'][refIndex])
    return nLastVisitsDate    





In [8]:
encoderX = EncoderX('0-2-0')
encoderX.REPOSICAO_VALUE = 0.5

In [9]:

#referenceWeek = ['02/03/2019','03/03/2019','04/03/2019','05/03/2019','06/03/2019','07/03/2019','08/03/2019'] #spawna em tudo
dateRanges = pd.date_range('11-01-2018', '04-01-2019')
#dateRanges = pd.date_range('03-02-2019', '03-08-2019')
titles = []
finalTable = []
for refDate in dateRanges:
    refDate = datetimeToDateString(refDate) # depois tem que ter um formato de data so 
    for store in visits:
        dateRef = dateRefStoreByDate(visits[store],refDate)
        if len(dateRef) == 0:
            continue
        lastVisits = findLastVisits(dateRef)                     # ultimas visitas que aconteceram naquela loja
        if len(lastVisits) != 5:
            continue
        for product in visits[store]['info']['allProducts']:
            yProd = visits[store][refDate][product]
            yProd = encoderX.oneHot(yProd)
            if yProd[2] == 1:
                yPred = -1
            else:
                yPred = yProd[0]
            visitLine = list([store, product, dateStringToDatetime(refDate), yPred])
            for date in lastVisits:
                diffDate = (dateStringToDatetime(refDate) - dateStringToDatetime(date)).days
                xProd = visits[store][date][product]
                xProd = encoderX.oneHot(xProd)
                if xProd[0] == -1:
                    print(visits[store][date][product])
                    raise Exception('stop')
                xProd.append(diffDate)
                visitLine += xProd
            finalTable += [visitLine] 
finalTable =  pd.DataFrame(data=finalTable)              

In [10]:
columns = ['store', 'product', 'score_date','y',
                   'presenca1','estoque1','status1','datadist1',
                   'presenca2','estoque2','status2','datadist2',
                   'presenca3','estoque3','status3','datadist3',
                   'presenca4','estoque4','status4','datadist4',
                   'presenca5','estoque5','status5','datadist5']
finalTable.columns = columns
yDescontinuado = finalTable[finalTable.y == -1].index   # fui fazer a visita e o produto tinha sido descontinuado
finalTable = finalTable.drop(yDescontinuado)
finalTable = finalTable.reset_index(drop=True)

In [11]:
prodId = {}
k = 0
for prod in list(set(finalTable.loc[:,'product'])):
    prodId[prod] = k
    k += 1
allProdId = [prodId[prod] for prod in finalTable.loc[:,'product']]
finalTable['id'] = allProdId

In [12]:
collections.Counter(finalTable.y)

Counter({0.5: 13920, 0.0: 4925, 1.0: 34873})

In [13]:
finalTable.loc[3021,:]

store         Hiper ABC Santo Ant nio do Monte LOJA 31
product                    SUSPIRO BARBIERI LIM O 150G
score_date                         2018-12-21 00:00:00
y                                                    1
presenca1                                            1
estoque1                                             0
status1                                              0
datadist1                                            4
presenca2                                            1
estoque2                                             0
status2                                              0
datadist2                                            7
presenca3                                            1
estoque3                                             0
status3                                              0
datadist3                                           11
presenca4                                            1
estoque4                                             0
status4   

In [14]:
finalTable.to_csv('ruptura-barbieri-montagem-5-repo-05.csv',sep=';')

# GERANDO TABELA PARA CONFERENCIA

In [ ]:
# APENAS BARBIERI
indBarbieri = []
for i in data.index:
    if data.Produto[i] in todosBiscoitos:
        indBarbieri.append(i)
dataBarbieri = data.iloc[indBarbieri,:].copy()
dataBarbieri = dataBarbieri.reset_index(drop=True)
dataBarbieri.to_csv('data-barbieri.csv',sep=';')